# Spotify API Notebook

In [1]:
import chatbot_lib as lib
from chatbot_lib import User

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import json

SPOTIPY_CLIENT_ID = '28e288eb88f342c79ac8389d6ab32529'
with open('client.secret') as f:
    SPOTIPY_CLIENT_SECRET = f.read()
SPOTIPY_REDIRECT_URI = "http://localhost:9090/"


def refresh_token(username):

    scope = 'user-library-read user-top-read'

    if(SPOTIPY_CLIENT_SECRET):
        token = util.prompt_for_user_token( username, 
                                            scope, 
                                            client_id = SPOTIPY_CLIENT_ID, 
                                            client_secret = SPOTIPY_CLIENT_SECRET, 
                                            redirect_uri = SPOTIPY_REDIRECT_URI)
    else:
        print("ERROR CONNECTING TO SPOTIFY. Check client.secret file")
        quit()
    
    return token

In [3]:
token = refresh_token("emccoskymusic")

In [4]:
def hydrate_user(user: User):
    token = refresh_token(user.spotify_username)
    tracks = []
    track_ids = []
    artists = []
    artist_ids = []
    if token:
        sp = spotipy.Spotify(auth=token)
        
        results = sp.current_user_top_tracks()
        for item in results['items']:
            tracks.append("Title: {name} Artist: {artist}".format(name = item['name'], artist = item['artists'][0]['name']))
            track_ids.append(item['id'])
        
        results = sp.current_user_top_artists(time_range='long_term')
        for item in results['items']:
            artists.append("{artist}".format(artist = item['name']))
            artist_ids.append(item['id'])
    else:
        print("Can't get token for", user.spotify_username)
    user.song_likes = tracks
    user.artist_likes = artists
    user.song_ids = track_ids
    user.artist_ids = artist_ids
    return user

In [73]:
new_user = User("Ethan", "PranjalSatija", [], [], [], [], [])
user = hydrate_user(new_user)
print(user.song_likes)
print(user.artist_likes)

Couldn't read cache at: .cache-PranjalSatija


OSError: [WinError 10013] An attempt was made to access a socket in a way forbidden by its access permissions

In [5]:
new_user = User("Ethan", "emccoskymusic", [], [], [], [], [])
user = hydrate_user(new_user)
token = refresh_token(user.spotify_username)
if token:
        sp = spotipy.Spotify(auth=token)
        artist_seed_str = ','.join([str(x) for x in user.artist_ids])
        track_seed_str = ','.join([str(x) for x in user.song_ids])
    
        #print(artist_seed_str)
        #print(track_seed_str)

        results = sp.recommendations(seed_artists = [user.artist_ids[0]], seed_tracks = [user.song_ids[0]])
        #print(results)
        for track in results['tracks']:
            print("Title: {name} Artist: {artist}".format(name = track['name'], artist = track['artists'][0]['name']))

Title: Forests Artist: Tom Rosenthal
Title: Melody Noir Artist: Patrick Watson
Title: My Boy (feat. J. Cole) - Freestyle Artist: Wale
Title: My Nigga Just Made Bail Artist: Bas
Title: Fuji Films Artist: IshDARR
Title: Carry You Artist: Novo Amor
Title: About The Weather Artist: Tom Rosenthal
Title: Meditate (feat. J.I.D.) Artist: EARTHGANG
Title: Z4L (with Bari & Jay2) Artist: Smino
Title: Paint Artist: The Paper Kites
Title: Malibu Sleep Artist: Col3trane
Title: Tribe (with J. Cole) Artist: Bas
Title: Keep Me Warm Artist: Tom Rosenthal
Title: Proud Of U (feat. Young Thug) Artist: EARTHGANG
Title: FALL BACK Artist: Duckwrth
Title: LIFE Artist: Saba
Title: Swarm Swamp Swim Artist: Tom Rosenthal
Title: Amphetamine Artist: Smino
Title: OooWee Artist: Rapsody
Title: Lighthouse Artist: Patrick Watson


In [67]:
def get_recommendations(user: User):
    token = refresh_token(user.spotify_username)
    recs = []
    if token:
            sp = spotipy.Spotify(auth=token)
            results = sp.recommendations(seed_artists = [user.artist_ids[0]], seed_tracks = [user.song_ids[0]])
            #print(results)
            for track in results['tracks']:
                recs.append("Title: {name} Artist: {artist}".format(name = track['name'], artist = track['artists'][0]['name']))
    else:
        print("Can't get token for", user.spotify_username)
    return recs

In [70]:
import random
new_user = User("Ethan", "emccoskymusic", [], [], [], [], [])
user = hydrate_user(new_user)
recs = get_recommendations(user)
random.shuffle(recs)
print(recs)

['Title: P.A.S.T.A Artist: Tom Rosenthal', 'Title: Off Deez (with J. Cole) Artist: JID', 'Title: Rotation 112th (feat. Rich The Kid) - Remix Artist: Jay Rock', 'Title: Je te laisserai des mots Artist: Patrick Watson', 'Title: Days Like This (feat. Khalid) Artist: GoldLink', 'Title: Front Desk Artist: Bas', 'Title: Jim and Dwight Artist: Tom Rosenthal', 'Title: Forget Tomorrow Artist: Mighty Oaks', 'Title: I Might Need Security Artist: Chance the Rapper', 'Title: Places + Faces Artist: Aminé', 'Title: Prove It (feat. J. Cole) Artist: Big K.R.I.T.', 'Title: Zendaya (feat. J. Cole) Artist: Cozz', 'Title: My Life Is A Song For You Artist: Tom Rosenthal', 'Title: Workin Out Artist: JID', 'Title: Breathe Again Artist: Harrison Storm', 'Title: Reicht dir das Artist: Provinz', 'Title: If We All Die Tomorrow Artist: Tom Rosenthal', 'Title: Cold Artist: Novo Amor', 'Title: Arms Artist: The Paper Kites', 'Title: Come To Me Artist: ÄTNA']
['Title: My Life Is A Song For You Artist: Tom Rosenthal', 